# Final Data

Goal: Combine yields, securities and options data to create a single data set of features and labels used in the multilayer perceptron model

In [1]:
import pandas as pd

In [2]:
%cd '/Users/benjochem/Desktop/Junior/Research'

/Users/benjochem/Desktop/Junior/Research


In [3]:
yields = pd.read_csv('Project/data/interim/fixed_yields.csv')
options = pd.read_csv('Project/data/interim/full_options_w_dates_TTE.csv')
securities = pd.read_csv('Project/data/interim/securities_w_date.csv')

In [4]:
yields.head()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,date
0,0.05,NaN,0.08,0.18,0.45,1.09,1.66,2.65,3.36,3.85,4.60,4.65,20100104
1,0.03,NaN,0.07,0.17,0.41,1.01,1.57,2.56,3.28,3.77,4.54,4.59,20100105
2,0.03,NaN,0.06,0.15,0.40,1.01,1.60,2.60,3.33,3.85,4.63,4.70,20100106
3,0.02,NaN,0.05,0.16,0.40,1.03,1.62,2.62,3.33,3.85,4.62,4.69,20100107
4,0.02,NaN,0.05,0.15,0.37,0.96,1.56,2.57,3.31,3.83,4.61,4.70,20100108


In [5]:
options.head()

,cp_flag,strike_price,volume,open_interest,delta,gamma,impl_volatility,best_bid,best_offer,Date,exDate,TTE
0,C,15000,0,0,NaN,NaN,NaN,13.3,13.8,20100528,20100619,22
1,C,16000,0,0,NaN,NaN,NaN,12.3,12.8,20100528,20100619,22
2,C,17000,0,0,NaN,NaN,NaN,11.3,11.8,20100528,20100619,22
3,C,18000,0,0,NaN,NaN,NaN,10.3,10.8,20100528,20100619,22
4,C,19000,0,0,NaN,NaN,NaN,9.3,9.8,20100528,20100619,22


In [6]:
securities.head()

,close,volume,return,Date,vol_20
0,104.58,215700,NaN,20090130,NaN
1,104.25,308571,-0.003155,20090202,NaN
2,99.37,177709,-0.046811,20090203,NaN
3,99.74,156854,0.003723,20090204,NaN
4,99.13,243819,-0.006116,20090205,NaN


In [7]:
#find number of unique dates in options
#options with the same date will be associated with the same row in the treasury yields data 
dates_in_options=set(options.Date.values)

In [8]:
#find options that do not have a corresponding row for treasury yields at their date
not_in_treasury=dates_in_options - set(yields.date.values)

In [9]:
# all options dates that do not have corresponding treasury yields
not_in_treasury

{20101111,
 20111010,
 20111111,
 20121008,
 20121112,
 20131014,
 20131111,
 20141013,
 20141111,
 20151012,
 20151111,
 20161010,
 20161111,
 20171009,
 20181008,
 20181112,
 20191014,
 20191111}

In [10]:
#create a dictionary from the treasury yields
# date as key, dictionary of the different rates(based on maturity) as value
#each date in yields will be a nested dictionary in yields_dictionary
yields_dictionary = {}
for i, row in yields.iterrows(): 
    d = dict(row)                                 # create a dictionary of the current row in the yields data
    yields_dictionary[int(d['date'])] = d         # add dictionary d to yields_dictionary with the date of the row as the key and the entire row as the value
    del yields_dictionary[int(d['date'])]['date'] # drop date from the nested dictionary value since it is used as the key

In [11]:
#create a set of all dates in yields data
yield_dates= set(yields.date.values)

In [12]:
#create data frame that includes ONLY options that have yield data
options_w_yields=options[options.Date.isin(yield_dates)]

In [13]:
# how many total options have corresponding yields data
len(options_w_yields)

3121129

In [14]:
def find_t_yield(row):
    
        date = row.Date #date value for the option on current row
        tte=row.TTE #time to expiration for the option on current row
        # detrmine which t-yield rate to use based on TTE
        
        if tte <= 45:
            length = '1 Mo'
        elif tte <= 75:
            length = '2 Mo'
        elif tte <= 135:
            length = '3 Mo'
        elif tte <= 270:
            length = '6 Mo'
        elif tte <= 547:
            length = '1 Yr'
        elif tte <= 912:
            length = '2 Yr'
        elif tte <= 1460:
            length = '3 Yr'
        elif tte <= 2190:
            length = '5 Yr'
        elif tte <= 3102:
            length = '7 Yr'
        elif tte <= 3975:
            length= '10 Yr'
        elif tte <= 6625:
            length = '20 Yr'
        else:
            length = '30 Yr'
            
        rate = yields_dictionary[date][length] #find and select the correct rate in date yields dictionary (go to correct date first then go to correct rate becasue it is nested)
     
        if rate >= 0: #2 month rates are missing for most of options (didnt start until 2018) so use 3 month rate if rate is not found or nan
            pass
        else:
            rate = yields_dictionary[date]['3 Mo']
    
        return rate 

In [15]:
options_w_yields['treasury_rate'] = options_w_yields.apply(lambda row: find_t_yield(row), axis=1)
options_w_yields

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cp_flag,strike_price,volume,open_interest,delta,gamma,impl_volatility,best_bid,best_offer,Date,exDate,TTE,treasury_rate
0,C,15000,0,0,NaN,NaN,NaN,13.30,13.80,20100528,20100619,22,0.15
1,C,16000,0,0,NaN,NaN,NaN,12.30,12.80,20100528,20100619,22,0.15
2,C,17000,0,0,NaN,NaN,NaN,11.30,11.80,20100528,20100619,22,0.15
3,C,18000,0,0,NaN,NaN,NaN,10.30,10.80,20100528,20100619,22,0.15
4,C,19000,0,0,NaN,NaN,NaN,9.30,9.80,20100528,20100619,22,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147370,P,53000,0,2,-0.737178,0.025131,0.793867,38.30,39.70,20191231,20220121,752,1.58
3147371,P,6000,0,9,-0.078954,0.008798,0.775066,0.25,1.81,20191231,20220121,752,1.58
3147372,P,7000,2,0,-0.100400,0.010908,0.748441,0.52,2.13,20191231,20220121,752,1.58
3147373,P,8000,2,12,-0.124924,0.011624,0.822367,1.40,2.77,20191231,20220121,752,1.58


In [16]:
#re-order columns to be used
options_w_yields=options_w_yields[['Date','exDate','cp_flag','strike_price','volume','treasury_rate','TTE','open_interest','best_bid','best_offer']]
options_w_yields

,Date,exDate,cp_flag,strike_price,volume,treasury_rate,TTE,open_interest,best_bid,best_offer
0,20100528,20100619,C,15000,0,0.15,22,0,13.30,13.80
1,20100528,20100619,C,16000,0,0.15,22,0,12.30,12.80
2,20100528,20100619,C,17000,0,0.15,22,0,11.30,11.80
3,20100528,20100619,C,18000,0,0.15,22,0,10.30,10.80
4,20100528,20100619,C,19000,0,0.15,22,0,9.30,9.80
...,...,...,...,...,...,...,...,...,...,...
3147370,20191231,20220121,P,53000,0,1.58,752,2,38.30,39.70
3147371,20191231,20220121,P,6000,0,1.58,752,9,0.25,1.81
3147372,20191231,20220121,P,7000,2,1.58,752,0,0.52,2.13
3147373,20191231,20220121,P,8000,2,1.58,752,12,1.40,2.77


In [17]:
securities=pd.read_csv('Project/data/interim/securities_w_date.csv')
securities

,close,volume,return,Date,vol_20
0,104.58,215700,NaN,20090130,NaN
1,104.25,308571,-0.003155,20090202,NaN
2,99.37,177709,-0.046811,20090203,NaN
3,99.74,156854,0.003723,20090204,NaN
4,99.13,243819,-0.006116,20090205,NaN
...,...,...,...,...,...
2994,14.94,10315489,-0.009940,20191224,0.032162
2995,14.92,13591197,-0.001339,20191226,0.032167
2996,15.23,26440757,0.020777,20191227,0.032183
2997,15.76,34261087,0.034800,20191230,0.030814


In [18]:
# drop securities with duplicate dates. Will result in having duplicate rows in final features data
print(len(securities))
securities=securities.drop_duplicates(subset='Date')
print(len(securities))

2999
2741


In [19]:
features = options_w_yields.merge(securities, on = 'Date', how = 'left')

In [20]:
print(len(features))
features.head()

3121129


,Date,exDate,cp_flag,strike_price,volume_x,treasury_rate,TTE,open_interest,best_bid,best_offer,close,volume_y,return,vol_20
0,20100528,20100619,C,15000,0,0.15,22,0,13.3,13.8,28.58,33996602,0.008825,0.072646
1,20100528,20100619,C,16000,0,0.15,22,0,12.3,12.8,28.58,33996602,0.008825,0.072646
2,20100528,20100619,C,17000,0,0.15,22,0,11.3,11.8,28.58,33996602,0.008825,0.072646
3,20100528,20100619,C,18000,0,0.15,22,0,10.3,10.8,28.58,33996602,0.008825,0.072646
4,20100528,20100619,C,19000,0,0.15,22,0,9.3,9.8,28.58,33996602,0.008825,0.072646


In [21]:
features = features.rename({'close':'sec_price','volume_y':'sec_volume','return':'sec_return','volume_x':'option_volume'}, axis = 1)
features.head()

,Date,exDate,cp_flag,strike_price,option_volume,treasury_rate,TTE,open_interest,best_bid,best_offer,sec_price,sec_volume,sec_return,vol_20
0,20100528,20100619,C,15000,0,0.15,22,0,13.3,13.8,28.58,33996602,0.008825,0.072646
1,20100528,20100619,C,16000,0,0.15,22,0,12.3,12.8,28.58,33996602,0.008825,0.072646
2,20100528,20100619,C,17000,0,0.15,22,0,11.3,11.8,28.58,33996602,0.008825,0.072646
3,20100528,20100619,C,18000,0,0.15,22,0,10.3,10.8,28.58,33996602,0.008825,0.072646
4,20100528,20100619,C,19000,0,0.15,22,0,9.3,9.8,28.58,33996602,0.008825,0.072646


In [23]:
features = features[['Date','exDate','cp_flag','TTE','strike_price','treasury_rate','sec_price','option_volume','open_interest','sec_volume','sec_return','vol_20','best_bid','best_offer']]
features.head()

,Date,exDate,cp_flag,TTE,strike_price,treasury_rate,sec_price,option_volume,open_interest,sec_volume,sec_return,vol_20,best_bid,best_offer
0,20100528,20100619,C,22,15000,0.15,28.58,0,0,33996602,0.008825,0.072646,13.3,13.8
1,20100528,20100619,C,22,16000,0.15,28.58,0,0,33996602,0.008825,0.072646,12.3,12.8
2,20100528,20100619,C,22,17000,0.15,28.58,0,0,33996602,0.008825,0.072646,11.3,11.8
3,20100528,20100619,C,22,18000,0.15,28.58,0,0,33996602,0.008825,0.072646,10.3,10.8
4,20100528,20100619,C,22,19000,0.15,28.58,0,0,33996602,0.008825,0.072646,9.3,9.8


In [24]:
features.to_csv('Project/data/interim/features.csv', index = False)